# Zero CO2

https://zeroco2.eco/en/projects/

In [ ]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
with open('../input/ZeroCo2/ZeroCo2.json', 'r') as file:
    data = json.load(file)
print(data[0])

In [ ]:
if isinstance(data, list) and isinstance(data[0], dict):
    df = pd.DataFrame(data)
    df.info()

In [ ]:
df.head(1)

### Fix geometries

In [ ]:
df['geometry'] = df['gps_position.coordinates'].apply(lambda x: Point(x))

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = 'EPSG:4326')
gdf['geometry'] = gdf['geometry'].make_valid()

### Harmonize nomenclature

In [ ]:
gdf = gdf[['project_id', 'id', 'description', 'countryName', 'date_created', 'geometry']]

In [ ]:
gdf['url'] = 'https://zeroco2.eco/en/projects/'
gdf['host_name'] = 'Zero CO2'

In [ ]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'project_id': 'project_id_reported',
     'id': 'site_id_reported',
    'description': 'project_description_reported',
    'countryName': 'country',
    'date_created' : 'planting_date_reported'
}
gdf.rename(columns=columns_rename_mapping, inplace=True)

In [ ]:
gdf['planting_date_reported'] = pd.to_datetime(gdf['planting_date_reported']).dt.year

In [ ]:
gdf = gdf.assign(site_sqkm=None,species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,country=None)
gdf.info()

### Save it

In [ ]:
gdf.to_file('../midsave/zero_co2.gpkg')